<a href="https://colab.research.google.com/github/Masum06/Indigenous-NMT/blob/main/notebooks/pointer_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Masum06/Indigenous-NMT.git

Cloning into 'Indigenous-NMT'...
remote: Enumerating objects: 272, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 272 (delta 21), reused 58 (delta 20), pack-reused 212
Receiving objects: 100% (272/272), 97.63 MiB | 31.53 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [2]:
cd Indigenous-NMT

/content/Indigenous-NMT


In [3]:
import pandas as pd
import os

In [4]:
df = pd.read_csv('data/bangla_chakma.csv').dropna().sample(frac=1)
df

,Bangla,Chakma
5088,দাস,গাভুর্
8729,ম্যাচ,সলা
9232,শক্তিসামর্থ্য,রত্বল্
9379,শীঘ্র,হাক্কে
9096,লম্বা,অজল্
...,...,...
4892,থই পাওয়া,থক্ পানা
4460,ঢলে পড়া,ঢলি পরানা
2805,গলানো,ঘলানা
2897,গিরিকন্দর,ধুরি


## For char training

In [ ]:
def split(s):
  s = s.replace(" ", "_")
  s = ' '.join(list(s))
  return s

In [ ]:
def merge(s):
  s = s.replace('\n', '_')
  s = ''.join(s.split())
  s = s.replace("_", ' ')
  return s

In [ ]:
df['Bangla'] = df['Bangla'].apply(lambda x: split(x))
df['Chakma'] = df['Chakma'].apply(lambda x: split(x))
df

,Bangla,Chakma
4940,দ খ ল _ ক র ে _ র া খ া,জ ু র ি _ থ ন া
1619,ও,আ
4030,জ ো র _ ক র ে,ব ল ে ব ল ে
249,অ ন ি য় ম,অ গ র ্ _ ব ে গ র ্
8228,ম ধ ্ য ে,ভ ি দ ি র ে
...,...,...
1728,ক ন ি ষ ্ ঠ,চ ি গ ো ন ্
8427,ম া ন ু ষ,ম ু ন ি চ ্ ছ ো র ্
3427,চ া ষ া ব া দ,চ া চ ্ ব া চ ্
4430,ড ি ঙ া ন ো,দ ে গ া ন া


## Write data to file

In [5]:
data = {
    "train": df,
    "test": df.sample(1000),
    "valid": df.sample(1000)
}

In [6]:
!rm data/*.txt

rm: cannot remove 'data/*.txt': No such file or directory


In [7]:
for key in data.keys():
  fp_input = open("data/src_{}.txt".format(key), 'w+')
  fp_input.write('\n'.join(list(data[key]['Bangla'].values)))
  fp_input.close

  fp_input = open("data/tgt_{}.txt".format(key), 'w+')
  fp_input.write('\n'.join(list(data[key]['Chakma'].values)))
  fp_input.close

In [8]:
!pip install OpenNMT-py

     |████████████████████████████████| 212 kB 8.3 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 76 kB 5.4 MB/s 
     |████████████████████████████████| 748.8 MB 18 kB/s 
     |████████████████████████████████| 58 kB 7.0 MB/s 
     |████████████████████████████████| 636 kB 54.5 MB/s 
     |████████████████████████████████| 14.3 MB 153 kB/s 
     |████████████████████████████████| 73 kB 2.2 MB/s 
     |████████████████████████████████| 1.2 MB 55.0 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0
  Attem

In [9]:
!export CUDA_VISIBLE_DEVICES

### Train with Copy Mechanism

In [10]:
!mkdir -p ./models

In [11]:
import numpy as np
measurer = np.vectorize(len)
measurer(df.values.astype(str)).max(axis=0)

array([26, 40])

In [12]:
!python OpenNMT-py-legacy/preprocess.py -train_src data/src_train.txt -train_tgt data/tgt_train.txt \
-valid_src data/src_valid.txt -valid_tgt data/tgt_valid.txt -src_seq_length 3 -tgt_seq_length 3 -save_data data/preprocessed_data

# -dynamic_dict 

[2021-07-31 03:51:14,975 INFO] Extracting features...
[2021-07-31 03:51:14,978 INFO]  * number of source features: 0.
[2021-07-31 03:51:14,979 INFO]  * number of target features: 0.
[2021-07-31 03:51:14,979 INFO] Building `Fields` object...
[2021-07-31 03:51:14,979 INFO] Building & saving training data...
[2021-07-31 03:51:15,001 INFO] Building shard 0.
[2021-07-31 03:51:15,282 INFO]  * saving 0th train data shard to data/preprocessed_data.train.0.pt.
[2021-07-31 03:51:15,499 INFO]  * tgt vocab size: 4963.
[2021-07-31 03:51:15,506 INFO]  * src vocab size: 4964.
[2021-07-31 03:51:15,532 INFO] Building & saving validation data...
[2021-07-31 03:51:15,557 INFO] Building shard 0.
[2021-07-31 03:51:15,569 INFO]  * saving 0th valid data shard to data/preprocessed_data.valid.0.pt.


In [60]:
!python OpenNMT-py-legacy/train.py \
-encoder_type brnn \
-global_attention mlp \
-world_size 1 -gpu_ranks 0 \
-train_steps 100000 -save_checkpoint_steps 10000 \
-valid_steps 5000 \
-data data/preprocessed_data \
-save_model ./models/bn_ck
-train_from /models/bn_ck_step_25000.pt \

[2021-07-31 04:26:13,545 INFO]  * src vocab size = 4964
[2021-07-31 04:26:13,546 INFO]  * tgt vocab size = 4963
[2021-07-31 04:26:13,546 INFO] Building model...
[2021-07-31 04:26:16,474 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(4964, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 250, num_layers=2, dropout=0.3, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(4963, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3, inplace=False)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_context): Linear(in_features=500, out_features=5

NameError: ignored

#Test

In [61]:
best_model_dir = "./models/bn_ck_step_100000.pt"

In [62]:
!python OpenNMT-py-legacy/translate.py -model $best_model_dir -src \
data/src_test.txt -output data/pred.txt -replace_unk -verbose -beam_size 1

Streaming output truncated to the last 5000 lines.
SENT 1: ['সাধ্য']
PRED 1: সেদাম্
PRED SCORE: 0.0000

[2021-07-31 04:46:39,150 INFO] 
SENT 2: ['নগ্ন']
PRED 2: এরেয়ে
PRED SCORE: -0.0081

[2021-07-31 04:46:39,150 INFO] 
SENT 3: ['উপশম']
PRED 3: সচ্রম্
PRED SCORE: 0.0000

[2021-07-31 04:46:39,150 INFO] 
SENT 4: ['সেভাবে']
PRED 4: সেজান্
PRED SCORE: 0.0000

[2021-07-31 04:46:39,150 INFO] 
SENT 5: ['বাখারি']
PRED 5: কপ্
PRED SCORE: 0.0000

[2021-07-31 04:46:39,151 INFO] 
SENT 6: ['সজারু']
PRED 6: কুদুক্
PRED SCORE: 0.0000

[2021-07-31 04:46:39,151 INFO] 
SENT 7: ['পরীক্ষিত']
PRED 7: বাজেয়ে
PRED SCORE: 0.0000

[2021-07-31 04:46:39,151 INFO] 
SENT 8: ['বাকপটু']
PRED 8: কদঘি
PRED SCORE: 0.0000

[2021-07-31 04:46:39,151 INFO] 
SENT 9: ['অগত্যা']
PRED 9: অতিত্তি
PRED SCORE: 0.0000

[2021-07-31 04:46:39,151 INFO] 
SENT 10: ['পিষ্ট']
PRED 10: কুর্বো
PRED SCORE: -0.0000

[2021-07-31 04:46:39,151 INFO] 
SENT 11: ['লাঙ্গল']
PRED 11: নাঙল্
PRED SCORE: 0.0000

[2021-07-31 04:46:39,151 INFO] 
SENT 12:

In [63]:
f = open("data/pred.txt")
lines = f.readlines()
f.close()

In [64]:
pred = pd.DataFrame({'pred':lines})

In [65]:
len(pred)

1000

In [66]:
test = data['test']

In [67]:
len(data['test'])

1000

In [68]:
pred

,pred
0,সেদাম্\n
1,এরেয়ে\n
2,সচ্রম্\n
3,সেজান্\n
4,কপ্\n
...,...
995,চিত্দিঘোল্\n
996,ঘিলে জবর্\n
997,গুলানা\n
998,গির্গিরানি\n


In [69]:
test[['Chakma']]

,Chakma
9999,আদাঙা
5493,কচদর
1321,সচ্রম্
10234,সেজান্
7254,কপ্
...,...
880,চিত্দিঘোল্
5158,রম়চক্র
8343,গুলানা
1887,তিত্তিরানা


In [33]:
def trim(s):
  # return ' '.join(s.split()) # char
  return s.strip() # word

In [70]:
import numpy as np

In [71]:
# Char
test_np = test['Chakma'].apply(trim).to_numpy()
pred_np = pred['pred'].apply(trim).to_numpy()

In [72]:
correct = 1*(test_np == pred_np)

In [73]:
correct.sum()/len(correct)

0.527

## Single data Test

In [54]:
## Char model

def test(nl):
  nl = '\n'.join(nl.split())
  nl = split(nl)
  nl = nl.replace('\n ', '\n')
  with open("test.txt", "w") as f:
    f.write(nl)
  
  # !python OpenNMT-py-legacy/translate.py -model $best_model_dir -src \
  # test.txt -output pred.txt -replace_unk -beam_size 1

  out = open("pred.txt", encoding='utf-8').read()
  out.replace('\n', '_')
  out = merge(out)
  print(out)


In [74]:
punk_list = ['।', '?', '!', '.']

In [79]:
def test(nl):
  original = nl
  nl = '\n'.join(nl.split())
  punk = ""
  if nl[-1] in punk_list:
    punk = nl[-1]
    nl = nl[:-1]
  # nl = split(nl)
  nl = nl.replace('\n ', '\n')
  with open("test.txt", "w") as f:
    f.write(nl)
  
  !python OpenNMT-py-legacy/translate.py -model $best_model_dir -src \
  test.txt -output pred.txt -replace_unk -beam_size 1

  out = open("pred.txt", encoding='utf-8').read()
  # out.replace('\n', ' ')
  # out = merge(out)
  print()
  out = ' '.join(out.split()) + punk
  print("Original sentence:", original)
  print("Translation:", out)


In [81]:
nl = """আমি ভালো আছি"""
test(nl)

[2021-07-31 05:17:06,485 INFO] Translating shard 0.
[2021-07-31 05:17:06,502 INFO] PRED AVG SCORE: -1.8301, PRED PPL: 6.2344

Original sentence: আমি ভালো আছি
Translation: মুই ভালা
